In [1]:
from kan import *
import pandas as pd
import numpy as np
import torch

In [2]:
import os
os.chdir("..")
os.getcwd()

'd:\\Documents\\GitHub\\crux-inductions\\Task 1'

In [3]:
# Load the iris dataset and prepare it for KAN model

df = pd.read_csv("Dataset/iris.csv")
df["variety"].replace(["Setosa", "Versicolor", "Virginica"], [0., 1., 2.], inplace=True) #replace the categorical labels with numbers
df = df.sample(frac=1, random_state=0).reset_index(drop=True) #shuffle the dataset

X = np.array(df.iloc[:, :-1].values)
Y = np.array(df.iloc[:, -1].values)
X = torch.from_numpy(X).float() # KAN needs its data in torch.tensor dtype
Y = torch.from_numpy(Y).type(torch.LongTensor) # CrossEntropyLoss needs the labels to be in integer dtype

In [4]:
train_split = 0.85

X_train = X[:int(len(X)*train_split)]
Y_train = Y[:int(len(Y)*train_split)]
X_test = X[int(len(X)*train_split):]
Y_test = Y[int(len(Y)*train_split):]

dataset = {}
dataset["train_input"] = X_train
dataset["test_input"] = X_test
dataset["train_label"] = Y_train
dataset["test_label"] = Y_test

In [5]:
"""
KAN model with 3 hidden layers of width 4, 5, and 3 respectively, grid size of 3, and k=3
"""
model = KAN(width=[4,5,3], grid=3, k=3, seed=0)

In [6]:
"""
define training and testing accuracy functions which will be used as metrics for KAN.train() method
"""

def train_acc():
    return torch.mean((torch.argmax(model(dataset['train_input']), dim=1) == dataset['train_label']).float())

def test_acc():
    return torch.mean((torch.argmax(model(dataset['test_input']), dim=1) == dataset['test_label']).float())

# train the model
results = model.train(dataset, opt="LBFGS", steps=20, metrics=(train_acc, test_acc), loss_fn=torch.nn.CrossEntropyLoss());

description:   0%|                                                           | 0/20 [00:00<?, ?it/s]c:\Users\trail\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
train loss: 2.27e-04 | test loss: 5.52e+00 | reg: 4.91e+02 : 100%|██| 20/20 [00:12<00:00,  1.64it/s]


In [7]:
#print results
print(f"Train Accuracy: {results['train_acc'][-1]}")
print(f"Test Accuracy: {results['test_acc'][-1]}")

Train Accuracy: 1.0
Test Accuracy: 0.9130434989929199


In [16]:
"""
Pykan also allows us to see the symbolic formula of the model. In this case we have 3 formulas each corresponding to a class.
Each formula has 4 variables which correspond to our 4 features in the iris dataset. Plugging in the values of these variables will 
give us the probability of the input belonging to that class.

Note that the formulas generated are not the same as the ones used to generate the data. (ideal function) 
They are just and approximation of the function that the model has learned and is true upto some error threshold given by 
the Kolmogorov-Arnold representation theorem.
"""

symbols = ['x','x^2','x^3','x^4','exp','log','sqrt','tanh','sin','abs'] #symbols that can be used in the formula
model.auto_symbolic(lib=symbols) #generate the symbolic formula


formula1, formula2, formula3 = model.symbolic_formula()[0]
print(f"Formula 1 :{formula1}\n\nFormula 2:{formula2}\n\nFormula 3:{formula3}")

fixing (0,0,0) with log, r2=0.9991572499275208
fixing (0,0,1) with x^3, r2=0.8089573979377747
fixing (0,0,2) with sin, r2=0.998892068862915
fixing (0,0,3) with sin, r2=0.9976062178611755
fixing (0,0,4) with sin, r2=0.9994670748710632
fixing (0,1,0) with sin, r2=0.9999797344207764
fixing (0,1,1) with sin, r2=1.0000001192092896
fixing (0,1,2) with sin, r2=0.9999666810035706
fixing (0,1,3) with exp, r2=0.9998781681060791
fixing (0,1,4) with sin, r2=0.9995633363723755
fixing (0,2,0) with sin, r2=0.9999454021453857
fixing (0,2,1) with sin, r2=0.9999930262565613
fixing (0,2,2) with sqrt, r2=0.9999890923500061
fixing (0,2,3) with sin, r2=0.999992311000824
fixing (0,2,4) with x^2, r2=0.9999874830245972
fixing (0,3,0) with tanh, r2=0.9998378753662109
fixing (0,3,1) with sin, r2=0.9999805092811584
fixing (0,3,2) with tanh, r2=0.9999189972877502
fixing (0,3,3) with tanh, r2=0.9999874234199524
fixing (0,3,4) with sin, r2=0.9999833106994629
fixing (1,0,0) with x^2, r2=0.9976719617843628
fixing (1,0